In [1]:
!jupyter notebook --version

7.2.2


The first cell in this notebook seems to work around a bug where the global decimal arithmetic context keeps resetting to the default.

If this "fix" still does not work for you (and it very well may not, results do not seem to be consistent), then you should switch to using plain Python files and running the file from the command line, or using an editor such as PyCharm, VSCode, etc.

[See this bug report that has not been addressed yet:
https://github.com/jupyter/notebook/issues/5260]

To avoid this bug, I am going to always set the global context in each cell (that works, it's only that once the cell finishes running the global context gets reset)

### Arithmetic Contexts

Rounding modes: https://docs.python.org/3/library/decimal.html#rounding-modes

Let's first look at the default `decimal` context:

In [2]:
import decimal
from decimal import Decimal

In [3]:
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

As we can see the default precision is `28`, and the default rounding is `ROUND_HALF_EVEN`.

We can change the global context - and if we do, all our decimal arithmetic will be subject to the settings in that context until we change that context again.

In [4]:
ctx = decimal.getcontext()

In [5]:
ctx.prec = 5
decimal.getcontext()

Context(prec=5, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

And we can change the context again:

In [6]:
decimal.getcontext().prec = 6
decimal.getcontext()

Context(prec=6, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

Let's also change the rounding method:

In [7]:
decimal.getcontext().rounding = decimal.ROUND_HALF_UP
decimal.getcontext()

Context(prec=6, rounding=ROUND_HALF_UP, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

The context precision means that calculations involving `Decimal` objects will be limited to `6` siginificant digits.

But this does not mean that you cannot create `Decimal` numbers with only `6` significant digits:

In [8]:
decimal.getcontext().prec=6
d1 = Decimal('123.4567890')

In [9]:
d1

Decimal('123.4567890')

As you can see, the precision is maintained.

However if we perform a calculation, the calculation's intermediate and final results will be limited to `6` siginificant digits:

In [10]:
decimal.getcontext().prec=6
d1 + Decimal(1)

Decimal('124.457')

And since we changed the rounding method, we should no longer have Banker's rounding:

In [11]:
decimal.getcontext().prec=6
decimal.getcontext().rounding = decimal.ROUND_HALF_UP
round(Decimal('100.445'), 2)

Decimal('100.45')

Let's reset our global context to what it was before:

In [12]:
ctx = decimal.getcontext()
ctx.prec = 28
ctx.rounding = decimal.ROUND_HALF_EVEN

In [13]:
round(Decimal('100.445'), 2)

Decimal('100.44')

Sometimes we need to **temporarily** change our context. One way to do this is to store the context values we are going to change, modify the global context, perform the operations, and then reset the global context back to what it was before.

In [14]:
current_rounding = decimal.getcontext().rounding
decimal.getcontext().rounding = decimal.ROUND_HALF_UP
print(round(Decimal('123.445'), 2))
decimal.getcontext().rounding = current_rounding
print(round(Decimal('123.445'), 2))

123.45
123.44


Doing these steps is a little tedious.

Instead we can use a context manager provided by `decimal` to do this work of remembering the settings, chaning them, and then reverting to the original settings.

In [15]:
print('Before:', decimal.getcontext())
with decimal.localcontext() as ctx:
    ctx.prec = 6
    ctx.rounding = decimal.ROUND_HALF_UP
    print('Local context:', ctx)
    print('Inside context:', round(Decimal('123.445'), 2))
print('After:', decimal.getcontext())

Before: Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[Inexact, Rounded], traps=[InvalidOperation, DivisionByZero, Overflow])
Local context: Context(prec=6, rounding=ROUND_HALF_UP, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[Inexact, Rounded], traps=[InvalidOperation, DivisionByZero, Overflow])
Inside context: 123.45
After: Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[Inexact, Rounded], traps=[InvalidOperation, DivisionByZero, Overflow])


As you can see, using a local context with a context manager does not affect the global context.